### Import and Setup

In [1]:
!pip install unidecode

     |████████████████████████████████| 235 kB 11.7 MB/s 


In [2]:
import json
import re
from unidecode import unidecode
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
from collections import Counter

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"  

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

### Data

In [4]:
f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/drake.json')
drake = json.load(f)
f.close()

f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/tswift.json')
taylor = json.load(f)
f.close()

drake = [drake['songs'][i]['lyrics'] for i in range(len(drake['songs']))]
taylor = [taylor['songs'][i]['lyrics'] for i in range(len(taylor['songs']))]

taylor_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', taylor[i])).split('\n') for i in range(len(taylor))]
taylor_lyrics = [[unidecode(i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[i for i in taylor_lyrics[j] if i != ''] for j in range(len(taylor_lyrics))]

drake_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', drake[i])).split('\n') for i in range(len(drake))]
drake_lyrics = [[unidecode(i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[i for i in drake_lyrics[j] if i != ''] for j in range(len(drake_lyrics))]

taylor_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in taylor_lyrics]
drake_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in drake_lyrics]

drake_tokenized = [[word_tokenize(drake_lyrics[i][j]) for j in range(len(drake_lyrics[i]))] for i in range(len(drake_lyrics))]
taylor_tokenized = [[word_tokenize(taylor_lyrics[i][j]) for j in range(len(taylor_lyrics[i]))] for i in range(len(taylor_lyrics))]

drake_tokenized = [[[word.lower() for word in line] for line in song] for song in drake_tokenized]
taylor_tokenized = [[[word.lower() for word in line] for line in song] for song in taylor_tokenized]

drake_length = sum([[len(sent) for sent in song] for song in drake_tokenized], [])
taylor_length = sum([[len(sent) for sent in song] for song in taylor_tokenized], [])

drake_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in drake_tokenized], [])
taylor_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in taylor_tokenized], [])

taylor_vocab = sum(taylor_lyrics,[])
drake_vocab = sum(drake_lyrics,[])

def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

vocab = taylor_vocab + drake_vocab
vocab_counts = Counter(vocab)
vocab = unique(vocab)
vocab = ['<pad>','<unk>','<s>', '</s>'] + vocab

from torch.utils import data
import torch

# These IDs are reserved.
MAX_SENT_LENGTH = 30
MAX_SENT_LENGTH_PLUS_SOS_EOS = 32
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3
RARE_WORD_TRESHOLD = 0

vocab_counts['<pad>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<unk>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<s>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['</s>'] = RARE_WORD_TRESHOLD + 1

class TSTDataset(data.Dataset):
    def __init__(self, taylor_sentences, drake_sentences, vocab, vocab_counts, sampling=1.):
        self.taylor_sentences = taylor_sentences[:int(len(taylor_sentences) * sampling)]
        self.drake_sentences = drake_sentences[:int(len(drake_sentences) * sampling)]

        self.max_seq_length = MAX_SENT_LENGTH_PLUS_SOS_EOS
        self.vocab = vocab
        self.vocab_counts = vocab_counts

        self.v2id = {v : i for i, v in enumerate(self.vocab)}
        self.id2v = {val : key for key, val in self.v2id.items()}
    
    def __len__(self):
        return min(len(self.taylor_sentences), len(self.drake_sentences))
    
    def __getitem__(self, index):
        taylor_sent = self.taylor_sentences[index]
        taylor_len = len(taylor_sent) + 2   # add <s> and </s> to each sentence
        taylor_id = []
        for w in taylor_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            taylor_id.append(self.v2id[w])

        taylor_id = ([SOS_INDEX] + taylor_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - taylor_len))

        drake_sent = self.drake_sentences[index]
        drake_len = len(drake_sent) + 2   # add <s> and </s> to each sentence
        drake_id = []
        for w in drake_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            drake_id.append(self.v2id[w])

        drake_id = ([SOS_INDEX] + drake_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - drake_len))

        return torch.tensor(taylor_id), taylor_len, torch.tensor(drake_id), drake_len

dataset = TSTDataset(taylor_lyrics, drake_lyrics, vocab, vocab_counts)

In [5]:
test_pct = 0.2
valid_pct = 0.1

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset)*(1-test_pct)),len(dataset)-int(len(dataset)*(1-test_pct))])
valid_dataset, train_dataset = torch.utils.data.random_split(train_dataset, [int(len(dataset)*valid_pct),len(train_dataset)-int(len(dataset)*valid_pct)])

### Encoder

In [6]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, dropout=0.):
    """
    Inputs: 
      - `input_size`: an int representing the RNN input size.
      - `hidden_size`: an int representing the RNN hidden size.
      - `dropout`: a float representing the dropout rate during training. Note
          that for 1-layer RNN this has no effect since dropout only applies to
          outputs of intermediate layers.
    """
    super(Encoder, self).__init__()
    self.rnn = nn.GRU(input_size, hidden_size, num_layers=1, batch_first=True,
                      dropout=dropout, bidirectional=False)

  def forward(self, inputs, lengths, init_state=None):
    """
    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of source
          sentences.
      - `lengths`: a 1d-tensor of shape (batch_size,) representing the sequence
          lengths of `inputs`.

    Returns:
      - `outputs`: a 3d-tensor of shape
        (batch_size, max_seq_length, hidden_size).
      - `finals`: a 3d-tensor of shape (num_layers, batch_size, hidden_size).
      Hint: `outputs` and `finals` are both standard GRU outputs. Check:
      https://pytorch.org/docs/stable/nn.html#gru
    """
    # Our variable-length inputs are padded to the same length for batching
    # Here we "pack" them for computational efficiency (see note below)
    packed = pack_padded_sequence(inputs, lengths.cpu(), batch_first=True,
                                  enforce_sorted=False)
    outputs, finals = self.rnn(packed, init_state)
    outputs, _ = pad_packed_sequence(outputs, batch_first=True,
                                     total_length=MAX_SENT_LENGTH_PLUS_SOS_EOS)
    return outputs, finals

### Decoder

In [7]:
class GeneratorTransferredSampled(nn.Module):
  """Define standard linear + softmax generation step."""
  def __init__(self, hidden_size, vocab_size, src_embed, gamma=0.001):
    """
    Inputs:
      - `src_embed`: a 2d-tensor of shape (vocab_size, embed_size )
    """
    super(GeneratorTransferredSampled, self).__init__()
    self.proj = nn.Linear(hidden_size, vocab_size, bias=True)
    self.gamma = gamma
    self.logsoftmax = nn.LogSoftmax(dim = 2)
    self.src_embed = src_embed

  def embedding(self,x):
    return torch.matmul(x,self.src_embed.weight)
    
  def gumbel_softmax(self,logits, eps=1e-20):
    U = torch.rand(logits.shape).to(device)
    G = -torch.log(-torch.log(U + eps) + eps).to(device)
    return self.logsoftmax((logits + G) / self.gamma)

  def forward(self, x):
    logits = self.proj(x)
    prob = self.logsoftmax(logits)
    output = self.embedding(prob)
    word  = logits.argmax(dim = 2, keepdim = False)

    return output, prob, word

  def forward_gumbel(self, x):
    logits = self.proj(x)
    prob = self.gumbel_softmax(logits)
    output = self.embedding(prob)
    word  = logits.argmax(dim = 2, keepdim = False)

    return output, prob, word

In [14]:

class Decoder(nn.Module):
  """An RNN decoder + generator with GRU"""

  def __init__(self, input_size, hidden_size, max_len,generator, num_layers = 1, dropout=0.):
    """
      Inputs:
        - `input_size` , `hidden_size`, and `dropout` the same as in Encoder.
    """
    super(Decoder, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.rnn = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True,
                      dropout=dropout, bidirectional=False)
    self.generator = generator
    self.max_len = max_len
    self.dropout_layer = nn.Dropout(p=dropout)
    self.rnn_to_pre = nn.Linear(input_size + hidden_size,
                                hidden_size, bias=False)

  def forward_step(self, prev_embed, hidden):
    """Helper function for forward below:
       Perform a single decoder step (1 word).

       Inputs:
      - `prev_embed`: a 3d-tensor of shape (batch_size, 1, embed_size = vocab_size)
          representing the padded embedded word vectors at this step in training
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the current hidden state.

      Returns:
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
          representing the current decoder hidden state.
      - `output`: a 3d-tensor of shape (batch_size, max_len, vocab_size)
          representing the total generated outputs.
    """
    pre_output, hidden = self.rnn(prev_embed, hidden)
    pre_output = torch.cat([prev_embed, pre_output], dim=2)
    pre_output = self.dropout_layer(pre_output)
    pre_output = self.rnn_to_pre(pre_output)
    pre_output = torch.tanh(pre_output)

    return hidden, pre_output

    ### Your code here!
    pre_output, hidden = self.rnn(prev_embed, hidden)
    pre_output = torch.cat([prev_embed, pre_output], dim=2)
    pre_output = self.dropout_layer(pre_output)
    pre_output = self.rnn_to_pre(pre_output)
    pre_output = self.pre_activation(pre_output)

  def forward(self, input, encoder_finals,max_len, hidden=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, 1, embed_size)
          representing a batch of padded embedded word vectors of SOS . 
          If size is (batch_size,max_len, embed_size), then it is teacher forcing.
      - `encoder_finals`: a 3d-tensor of shape
          (num_enc_layers, batch_size, hidden_size) representing the final
          encoder hidden states used to initialize the initial decoder hidden
          states.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the value to be used to initialize the initial decoder hidden states.
          If None, then use `encoder_finals`.
      - `max_len`: an int representing the maximum decoding length.
      - `style`: TAYLOR_STYLE or DRAKE_STYLE

    Returns:
      - `hidden`: a 3d-tensor of shape
          (num_layers, batch_size, hidden_size) representing the final hidden
          state for each element in the batch.
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_len, hidden_size) representing the raw decoder
          outputs (before mapping to a `trg_vocab_size`-dim vector).
      - `logits_vectors`: a 3d-tensor of shape
          (batch_size, max_len, trg_vocab_size) representing the mapped decoder
          outputs.
      - `words`: a 3d-tensor of shape
          (batch_size, max_len, 1) representing output sentence and
          the corresponding word index (can be used for embedding)  
    """

    # Initialize decoder hidden state.
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)
    output_vectors = []
    logits_vectors = []
    words = []
    hidden_states = []
    hidden_states.append(hidden[-1][:,None,:])
    for i in range(max_len-1) :
      hidden, prev_output = self.forward_step(input,hidden)
      input, logits, output_word = self.generator.forward(prev_output)
      # input, logits, output_word = self.generator(prev_output)

      # input = torch.concat([input,torch.full(input.shape,style)], axis = -1)
      logits_vectors.append(logits)
      output_vectors.append(input)
      words.append(output_word)
      hidden_states.append(prev_output)

    outputs = torch.cat(output_vectors, dim =1)
    logits_vectors = torch.cat(logits_vectors,dim = 1)
    words = torch.cat(words, axis = -1)
    hidden_states = torch.cat(hidden_states, axis = 1)
    return hidden, outputs , logits_vectors, words, hidden_states

  def forward_teacher(self, input, encoder_finals, max_len=None, hidden=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size,max_len, embed_size)
          representing a batch of padded embedded word vectors of original 
          sentence and acts as  teacher forcing.

    Returns:
      - `hidden`: a 3d-tensor of shape
          (num_layers, batch_size, hidden_size) representing the final hidden
          state for each element in the batch.
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_len, hidden_size) representing the raw decoder
          outputs (before mapping to a `trg_vocab_size`-dim vector).
      - `logits_vectors`: a 3d-tensor of shape
          (batch_size, max_len, trg_vocab_size) representing the mapped decoder
          outputs each represents the probability? 
      - `words`: a 3d-tensor of shape
          (batch_size, max_len, 1) representing output sentence and
          the corresponding word index (can be used for embedding)      
    """

    # Initialize decoder hidden state.
    if max_len is None:
      max_len = input.shape[1]
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)
    output_vectors = []
    logits_vectors = []
    words = []
    hidden_states = []
    hidden_states.append(hidden[-1][:,None,:])
    for i in range(max_len):
      hidden, prev_output = self.forward_step(input[:,i:i+1,:],hidden)
      output, logits, output_word = self.generator(prev_output)
      
      logits_vectors.append(logits)
      output_vectors.append(output)
      words.append(output_word)
      hidden_states.append(prev_output)

    outputs = torch.cat(output_vectors, dim =1)
    logits_vectors = torch.cat(logits_vectors,dim = 1)
    words = torch.cat(words, axis = -1)
    hidden_states = torch.cat(hidden_states, axis = 1)
    return hidden, outputs , logits_vectors, words, hidden_states

  def init_hidden(self, encoder_finals):
    """Use encoder final hidden state to initialize decoder's first hidden
       state.

       Input: `encoder_finals` is same as in forward()

       Returns: 
         - `decoder_init_hiddens`: a 3d-tensor of shape 
              (num_layers, batch_size, hidden_size) representing the initial
              hidden state of the decoder for each element in the batch 
    """
    decoder_init_hiddens = torch.tanh(encoder_finals)
    return decoder_init_hiddens

In [17]:
### Work in progress
class EncoderDecoder(nn.Module):
  def __init__(self, encoder, decoder, line_embed, generator):
    super(EncoderDecoder, self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.line_embed = line_embed
    self.generator = generator

  def forward(self, lines, line_lens):
    encoder_hiddens, encoder_finals = self.encode(lines, line_lens)
    del encoder_hiddens
    return self.reconstruct(encoder_finals, lines[:, :-1]), self.decode(encoder_finals)
    # decode_orig = self.decode(h0_orig)
    # return rec_orig, decode_orig

  def encode(self, lines, line_lens):
    return self.encoder(self.line_embed(lines), line_lens)
    
  def reconstruct(self, h0, lines):
    original = self.line_embed(lines)
    return self.decoder.forward_teacher(original,h0)

  def decode(self, h0):
    target = self.line_embed(torch.tensor([SOS_INDEX]).repeat(h0.size()[1],1).to(device))
    return self.decoder.forward(target,h0,max_len)

epochs = 3
lr = 1e-3
batch_size = 32
print_every = 100
max_len = dataset.max_seq_length
vocab_size = len(vocab)
embed_size = 256
hidden_size = 256
dropout = 0.2
gamma = 0.001

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

line_embed = nn.Embedding(vocab_size, embed_size)
encoder = Encoder(embed_size,hidden_size)
generator = GeneratorTransferredSampled(hidden_size,vocab_size, line_embed, gamma = gamma)
decoder = Decoder(embed_size, hidden_size, max_len=vocab_size, generator = generator,dropout=dropout)
model = EncoderDecoder(encoder, decoder, line_embed, generator).to(device)
optimizer_model = torch.optim.Adam(model.parameters(), lr=lr)
rec_loss = nn.NLLLoss(reduction="mean",ignore_index = PAD_INDEX)

for epoch in range(epochs):
  epoch_rec_loss = 0
  epoch_tokens = 0
  model.train()
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(train_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)

    # Train model
    rec_orig,dec_orig  = model(lines, line_lens)    
    loss_rec = rec_loss(input=rec_orig[2].permute(0,2,1), target=lines[:, 1:])

    optimizer_model.zero_grad()
    loss_rec.backward()
    optimizer_model.step()
    
    epoch_rec_loss += loss_rec.item() * line_lens.sum().item()
    epoch_tokens += line_lens.sum().item()

  print("Finished Training Epoch ", epoch)
  print("Training PPL", np.exp(epoch_rec_loss / float(epoch_tokens)))
  val_loss = 0
  val_tokens = 0

  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(valid_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)

    rec_orig,dec_orig  = model(lines, line_lens)    
    loss_rec = rec_loss(input=rec_orig[2].permute(0,2,1), target=lines[:, 1:])

    val_loss += loss_rec.item() * line_lens.sum().item()
    val_tokens += line_lens.sum().item()
  
  print("Valid PPL", np.exp(val_loss / float(val_tokens)))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Finished Training Epoch  0
Training PPL 221.76302979114053
Valid PPL 112.68749254539021
Finished Training Epoch  1
Training PPL 75.18844053075715
Valid PPL 70.16073769462727
Finished Training Epoch  2
Training PPL 47.76703979213335
Valid PPL 54.0301458856892


In [21]:
rec_orig[3]

tensor([[ 8366,  8089,  4939, 12474,  7582,  1019,  2897,  9061,  2856,  8462,
          8366,  8089,  4939, 11497,  9061,   704,  8366, 10812,  1019, 12503,
          8089,  8089,  6419,  6419,  8089,  5860,  9061,     3,     3,     3,
             3],
        [ 8131,  4173,  4891,  7135,  8462,  8089,  8089,  4939,  7135,  7135,
          8462, 12487,  8462,  8366,  8131,  5860,  8447,  7135,   860,  8697,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3],
        [  609,  9061,  3277,  8447,  5370,  8447, 11524,  8462,   609, 11634,
         12474, 11073,  8089,  4830,  8089,  4939,  1019,  8462,     3,  8089,
          8089,  4939,     3,     3,     3,     3,     3,     3,     3,     3,
             3],
        [ 8366,  6310,  9946,  9946,  8462,  8366,  4479,   811,  8366,  8366,
          8366,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
 

In [18]:
def lookup_words(x, vocab):
  return [vocab[i] for i in x]

idx=5
print(lookup_words(lines[idx], vocab))
print(lookup_words(rec_orig[3][idx], vocab))
print(lookup_words(dec_orig[3][idx], vocab))

print(len(lookup_words(rec_orig[3][idx], vocab)))
print(len(lookup_words(dec_orig[3][idx], vocab)))

['<s>', 'rihanna', '-', 'close', 'to', 'you', ',', 'ariana', 'grande', '-', 'everyday', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['to', 'to', '``', 'to', ',', ',', 'and', 'to', 'to', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
['got', 'my', 'whole', 'whole', 'only', 'no', 'many', 'only', 'nice', 'sparks', 'only', 'whole', 'only', 'my', 'young', 'only', 'many', 'fuck', 'young', 'long', 'the', 'only', 'whole', 'many', 'only', 'enchanted', 'many', 'whole', 'nice', 'my', 'nice']
31
31


In [ ]:
def lookup_words(x, vocab):
  return [vocab[i] for i in x]

idx=5
print(lookup_words(lines[idx], vocab))
print(lookup_words(rec_orig[3][idx], vocab))
print(lookup_words(dec_orig[3][idx], vocab))

print(len(lookup_words(rec_orig[3][idx], vocab)))
print(len(lookup_words(dec_orig[3][idx], vocab)))

['<s>', 'i', "'d", 'hold', 'you', 'as', 'the', 'water', 'rushes', 'in', ',', 'if', 'i', 'could', 'dance', 'with', 'you', 'again', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['i', "'m", 'be', 'you', 'the', 'you', 'way', 'i', ',', 'the', 'i', 'you', "'m", 'be', ',', 'you', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
['i', 'party', 'championships', 'championships', 'magnet', 'truest', 'championships', 'case', 'biggie', 'contracts', 'this', 'mission', '11/9/2018', 'remote', 'drop-top', 'worn', 'witches', 'championships', 'avoidin', 'magnet', 'podrug', 'bleach', 'velcro', 'sparkly', 'mission', 'championships', 'championships', 'reclinin', 'vsekh', 'championships', 'forgiveness', 'toughen']
31
32


### Classifier

In [ ]:
class LSTMClassifier(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, LSTMlayers=1, dropout = 0.2):
    super(LSTMClassifier, self).__init__()
    self.embed = nn.Linear(vocab_size, embed_size)
    self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=LSTMlayers, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.fc1 =  nn.Linear(in_features=hidden_size, out_features=hidden_size+1)
    self.fc2 = nn.Linear(hidden_size+1, 1)

  def forward(self, seq):
    seq = self.embed(seq)
    output, (hidden,cell) = self.lstm(seq)
    output = self.dropout(output)
    output = self.fc1(output[:,-1])
    output = F.relu(output)
    output = self.fc2(output)
    return output
    return output.reshape(output.size(1), -1)

In [ ]:
class LSTMDiscriminator(nn.Module):
  def __init__(self, input_size, hidden_size, LSTMlayers=1, dropout = 0.2):
    super(LSTMDiscriminator, self).__init__()
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=LSTMlayers, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.fc1 =  nn.Linear(in_features=hidden_size, out_features=hidden_size+1)
    self.fc2 = nn.Linear(hidden_size+1, 1)

  def forward(self, seq):
    output, (hidden,cell) = self.lstm(seq)
    output = self.dropout(output)
    output = self.fc1(output[:,-1])
    output = F.relu(output)
    output = self.fc2(output)
    return output
    return output.reshape(output.size(1), -1)

### Training

In [ ]:
class TSTModel(nn.Module):
  def __init__(self, max_len, vocab_size, embed_size, hidden_size_z, hidden_size_y, line_embed, encoder, generator, decoder, classifier):
    super(TSTModel, self).__init__()

    self.hidden_size = hidden_size_y + hidden_size_z

    self.encoder = encoder
    self.generator = generator
    self.decoder = decoder
    self.classifier = classifier

    self.line_embed = line_embed
    self.y_embed_enc = nn.Embedding(2,hidden_size_y)
    self.y_embed_gen = nn.Embedding(2,hidden_size_y)

    self.max_len = max_len
    self.vocab_size = vocab_size
    self.embed_size = embed_size
    self.hidden_size_z = hidden_size_z
    self.hidden_size_y = hidden_size_y

  def forward(self, lines, line_lens, labels):
    encoded_lines = self.encode(lines, line_lens, labels)
    z = encoded_lines[1][-1][:,self.hidden_size_y:]

    h0_orig = torch.cat((self.y_embed_gen(labels),z), 1)[None,:]
    h0_tsf = torch.cat((self.y_embed_gen(1-labels),z), 1)[None,:]

    # Decode back into original form for reconstruction
    rec_orig = self.reconstruct(h0_orig, lines[:, :-1])

    # Decode into original and transferred forms for classification
 
    decode_orig = self.decode(h0_orig)
    decode_tsf = self.decode(h0_tsf)
    
    half = int(lines.size(0) / 2)

    discrim1_input = torch.cat((rec_orig[4][:half], decode_tsf[4][half:]))
    discrim0_input = torch.cat((rec_orig[4][half:], decode_tsf[4][:half]))

    classifier_lines = torch.cat((decode_orig[2], decode_tsf[2], F.one_hot(lines[:,1:], self.vocab_size).to(torch.float)), 0)
    pred_class = self.classifier(classifier_lines)

    # return rec_orig, decode_orig
    return rec_orig, pred_class, decode_orig, decode_tsf, discrim0_input, discrim1_input 

  def encode(self, lines, line_lens, labels):
    init_state = torch.cat((self.y_embed_enc(labels), torch.zeros((len(lines),self.hidden_size_z), device=device)), 1)[None,:].to(device)
    return self.encoder(self.line_embed(lines), line_lens, init_state)

  def reconstruct(self, h0, lines):
    original = self.line_embed(lines)
    return self.decoder.forward_teacher(original,h0)

  def decode(self, h0):
    target = self.line_embed(torch.tensor([SOS_INDEX]).repeat(h0.size()[1],1).to(device))
    return self.decoder.forward(target,h0,self.max_len)

In [ ]:
epochs = 4
class_epochs = 0
lr = 1e-3
batch_size = 32
print_every = 100

max_len = dataset.max_seq_length
vocab_size = len(vocab)
embed_size = 256
hidden_size_z = 256
hidden_size_y = 128
hidden_size = hidden_size_z + hidden_size_y
dropout = 0.2
gamma = 0.001

TAYLOR_STYLE=1 # for information only, don't change
DRAKE_STYLE=0  # for information only, don't change
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

line_embed = nn.Embedding(vocab_size, embed_size)
encoder = Encoder(embed_size,hidden_size)
generator = GeneratorTransferredSampled(hidden_size,vocab_size, line_embed, gamma = gamma)
decoder = Decoder(embed_size, hidden_size, max_len=vocab_size, generator = generator,dropout=dropout)
classifier = LSTMClassifier(embed_size, hidden_size_z, vocab_size, dropout =0.2)

discriminator0 = LSTMDiscriminator(hidden_size, hidden_size).to(device)
discriminator1 = LSTMDiscriminator(hidden_size, hidden_size).to(device)

model = TSTModel(dataset.max_seq_length, vocab_size, embed_size, hidden_size_z, hidden_size_y, line_embed, encoder, generator, decoder, classifier).to(device)
optimizer_model = torch.optim.Adam(model.parameters(), lr=lr) 
optimizer_discr = torch.optim.Adam(list(discriminator0.parameters()) + list(discriminator1.parameters()), lr=lr) 

# rec_loss = nn.CrossEntropyLoss(reduction="mean",ignore_index = PAD_INDEX)
rec_loss = nn.NLLLoss(reduction="mean",ignore_index = PAD_INDEX)
class_loss = nn.BCEWithLogitsLoss(reduction="mean")
discr_loss = nn.BCEWithLogitsLoss(reduction="mean")

epoch_losses = []
for epoch in range(epochs):
  epoch_loss = 0
  epoch_class_loss = 0
  epoch_rec_loss = 0
  epoch_adv_loss = 0
  epoch_loss_d = 0
  epoch_tokens = 0
  model.train()
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(train_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)
    
    fake_labels = torch.cat((torch.zeros(size=(len(taylor_lines),), dtype=torch.int32),torch.ones(size=(len(taylor_lines),),dtype=torch.int32))).to(device)
    fake_labels = fake_labels.unsqueeze(1)

    # Train discriminator

    # rec_orig, pred_class, decode_orig, decode_tsf, pred_fake0, pred_fake1 = model(lines, line_lens, labels)
    # pred_fake0 = discriminator0(pred_fake0)
    # pred_fake1 = discriminator1(pred_fake1)
    # loss_d0 = discr_loss(pred_fake0, fake_labels.to(torch.float))
    # loss_d1 = discr_loss(pred_fake1, fake_labels.to(torch.float))
    # loss_d = loss_d0 + loss_d1

    # optimizer_discr.zero_grad()
    # loss_d.backward()
    # optimizer_discr.step()

    # Train model

    rec_orig, pred_class, decode_orig, decode_tsf, pred_fake0, pred_fake1 = model(lines, line_lens, labels)

    # pred_fake0 = discriminator0(pred_fake0)
    # pred_fake1 = discriminator1(pred_fake1)

    loss_rec = rec_loss(input=rec_orig[2].permute(0,2,1), target=lines[:, 1:])
    loss_class = class_loss(pred_class, classifier_labels.to(torch.float))
    # loss_adv0 = class_loss(pred_fake0[len(drake_lines):], fake_labels[len(drake_lines):].to(torch.float))
    # loss_avd1 = class_loss(pred_fake1[len(taylor_lines):], fake_labels[len(taylor_lines):].to(torch.float))

    loss = loss_rec + loss_class
    # loss = loss_rec + loss_class - (loss_adv0 + loss_avd1)
    # loss = loss_rec

    optimizer_model.zero_grad()
    loss.backward()
    optimizer_model.step()
    
    epoch_loss += loss.item()
    # epoch_loss_d += loss_d.item()
    epoch_class_loss += loss_class.item()
    epoch_rec_loss += loss_rec.item() * line_lens.sum().item()
    epoch_tokens += line_lens.sum().item()
    # epoch_adv_loss += (loss_adv0.item() + loss_avd1.item())

    if model.training and i % print_every == 0:
      print("Epoch Step: %d Loss: %f" % (i, loss.item()))
  
  epoch_losses.append(epoch_loss)
  print("Finished Training Epoch ", epoch)
  print("Epoch Loss", epoch_loss)
  print("Classification Loss", epoch_class_loss)
  print("Training PPL", np.exp(epoch_rec_loss / float(epoch_tokens)))
  print("Adversarial Loss", epoch_adv_loss)
  print("Discriminator Loss", epoch_loss_d)
  val_loss = 0
  val_tokens = 0
  val_class_loss = 0
  correct_pred = 0
  correct_pred_all = 0

  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(valid_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)
    
    fake_labels = torch.cat((torch.zeros(size=(len(taylor_lines),), dtype=torch.int32),torch.ones(size=(len(taylor_lines),),dtype=torch.int32))).to(device)
    fake_labels = fake_labels.unsqueeze(1)

    rec_orig, pred_class, decode_orig, decode_tsf, pred_fake0, pred_fake1 = model(lines, line_lens, labels)
    loss_rec = rec_loss(input=rec_orig[2].permute(0,2,1), target=lines[:, 1:])
    loss_class = class_loss(pred_class, classifier_labels.to(torch.float))

    val_loss += loss_rec.item() * line_lens.sum().item()
    val_tokens += line_lens.sum().item()
    # val_class_loss += loss_class.item()*classifier_labels.size(0)

    correct_pred += torch.sum(1*(1*(torch.sigmoid(pred_class[-len(lines):])>=0.5) == labels.unsqueeze(1)))
    correct_pred_all += torch.sum(1*(1*(torch.sigmoid(pred_class)>=0.5) == classifier_labels))
  
  print("Valid PPL", np.exp(val_loss / float(val_tokens)))
  print("Valid Classification Loss", val_class_loss / float(len(valid_loader)*6))
  print("Classification Accuracy on Generated", correct_pred / (2.*len(valid_dataset)))
  print("Classification Accuracy on All", correct_pred_all / (3.*2.*len(valid_dataset)))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch Step: 0 Loss: 10.175980
Epoch Step: 100 Loss: 5.812898
Epoch Step: 200 Loss: 5.229771
Finished Training Epoch  0
Epoch Loss 1268.1461277008057
Classification Loss 146.50422883033752
Training PPL 194.30359768013915
Adversarial Loss 0
Discriminator Loss 0
Valid PPL 102.0429002991721
Valid Classification Loss 21.186714926073627
Epoch Step: 0 Loss: 5.206198
Epoch Step: 100 Loss: 4.831620
Epoch Step: 200 Loss: 4.839772
Finished Training Epoch  1
Epoch Loss 1033.8395166397095
Classification Loss 146.94581472873688
Training PPL 64.15095521318808
Adversarial Loss 0
Discriminator Loss 0
Valid PPL 56.711391119523014
Valid Classification Loss 21.753116400011123
Epoch Step: 0 Loss: 4.339231
Epoch Step: 100 Loss: 4.438923
Epoch Step: 200 Loss: 4.322330
Finished Training Epoch  2
Epoch Loss 922.5583679676056
Classification Loss 147.4784973859787
Training PPL 38.08967630460905
Adversarial Loss 0
Discriminator Loss 0
Valid PPL 42.72483216231617
Valid Classification Loss 21.139044161765806
Epoch 

In [ ]:
correct_pred = 0
correct_pred_all = 0
model.eval()
with torch.no_grad():
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(valid_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)

    rec_orig,pred_class, decode_orig,decode_tsf,d0,d1 =  model(lines, line_lens, labels)
    correct_pred += torch.sum(1*(1*(torch.sigmoid(pred_class[-len(lines):])>=0.5) == labels.unsqueeze(1)))
    correct_pred_all += torch.sum(1*(1*(torch.sigmoid(pred_class)>=0.5) == classifier_labels))
  print(correct_pred / (2.*len(valid_dataset)))
  print(correct_pred_all / (3.*2.*len(valid_dataset)))

tensor(0.8684, device='cuda:0')
tensor(0.6228, device='cuda:0')


In [ ]:
def lookup_words(x, vocab):
  return [vocab[i] for i in x]

with torch.no_grad():
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(train_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    # classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)

    # rec_orig,pred_class,decode_orig,decode_tsf,_,_ =  model(lines, line_lens, labels)

    rec_orig,pred_class, dec_orig,dec_tsf,d0,d1 =  model(lines, line_lens, labels)
    
    break


In [ ]:
idx=32
print(lookup_words(lines[idx], vocab))
print(lookup_words(rec_orig[3][idx], vocab))
print(lookup_words(dec_orig[3][idx], vocab))
print(lookup_words(dec_tsf[3][idx], vocab))

['<s>', 'i', "'m", 'blem', 'for', 'real', ',', 'i', 'might', 'just', 'say', 'how', 'i', 'feel', ',', 'do', "n't", 'switch', 'on', 'me', ',', 'i', 'got', 'big', 'plans', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['i', "'m", 'just', 'for', 'real', ',', 'i', 'do', 'do', 'do', 'i', 'i', "'m", ',', 'do', "n't", 'know', 'up', 'me', ',', 'i', 'do', 'ta', 'old', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
['i', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes']
['i', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swipes', 'swip